## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [ ]:
import os, sys
from pathlib import Path
from collab_splats.wrapper import Splatter, SplatterConfig
import pyvista as pv

pv.start_xvfb()

Set paths to the file for running splats

In [ ]:
base_dir = Path('/workspace/fieldwork-data/')
session_dir = base_dir / "birds/2024-02-06/SplatsSD"

# Make the configuration 
splatter_config = SplatterConfig(
    file_path=session_dir / "C0043.MP4",
    method='rade-features',
    frame_proportion=0.25, # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [ ]:
mesher_kwargs = {
    'depth_name': "median_depth",
    'depth_trunc': 1.0, # Should be between 1.0 and 3.0
    'voxel_size': 0.005, 
    'normals_name': "normals",
    'features_name': "distill_features", 
    'sdf_trunc': 0.03,
    'clean_repair': True,
    'align_floor': True,
}

splatter.mesh(
    mesher_type="Open3DTSDFFusion",
    mesher_kwargs=mesher_kwargs,
    # overwrite=True
)

### Plot the mesh!

We can use the splatter function ```plot_mesh``` to visualize given attributes of the mesh. The inherent attributes are RGB and Normals

In [ ]:
splatter.plot_mesh(attribute="RGB")

### Using semantic queries 

The mesh contains semantic features which we can query via positive and negative prompts. The goal of this is to find points that are more similar to the positive prompts compared to the negative prompts

In [ ]:
similarity = splatter.query_mesh(
    positive_queries=["tree"],
    negative_queries=["ground", "leaves"],
    output_fn="query-tree.ply"
)

Plot similarity maps

In [ ]:
splatter.plot_mesh(attribute=similarity, rgb=False)